In [ ]:
# Import Libraries
import time

from ImportData2D import get_label, getFilename, get_file_name
from stlSlicer import getSlicer, slicecoor, rotatestl
from imgSave import saveplot
import numpy as np

np.random.seed(125)


In [ ]:
label = getLabel("Occ_Sum", "median", True, False,False)
print(type(label[0]))
label = getLabel("Occ_Sum", "median", True, True,False)
print(type(label[0][0]))

In [ ]:
import os
test_name = "prep_test.tfrecords"
test_name = os.path.join("data", test_name)
print(test_name)

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print(type(train_images))

In [ ]:
i = 10
tffilename = "train"
img_addr = list([1,2,3,4])
print('Saving {} data: {}/{}'.format(tffilename,i, len(img_addr)))

In [ ]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
print(type(mnist))

In [ ]:
a = get_file_name(folder_name='../global_data/Original Format Data')
print((a))

In [9]:
A = ['a','b','c']
B = ['a','c', 'b']
diff = set(A).symmetric_difference(set(B))
diff2 = list(set(A) - set(B))
print(diff2)

[]


In [ ]:
try:
    raise NameError("Hello")
    A = 1
except NameError:
    B = 2
print(B)

In [2]:
A = [0,1,2,3,4,5,6,7,8,9]
index = 4
A.pop(index*2)
A.pop(index*2)
print(A) # Expect 6,7 to remove

[0, 1, 2, 3, 4, 5, 6, 7]


In [1]:
import os
A = './data/preparation_181_data_test_eval_address.txt'
print(os.path.abspath(A))
print(os.path.isfile(A))

/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/preparation_181_data_test_eval_address.txt
True


In [12]:
# define an empty list
error_file_names = []

# open file and read the content in a list
with open(A, 'r') as filehandle:  
    for line in filehandle:
        # remove linebreak which is the last character of the string
        current_name = line[:-1]
        # add item to the list
        error_file_names.append(ast.literal_eval(current_name))

In [15]:
print(error_file_names[0][1])

8


In [11]:
import ast
B = ast.literal_eval(error_file_names[0])
print(B[0])

/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_95111


In [15]:
A = [0,1,23,4,5,6,7,8,9,0]

print(range(5))
print(A[0:5])

range(0, 5)
[0, 1, 23, 4, 5]
